In [3]:
import os
import csv
from collections import defaultdict
import json

python -m birdnet_analyzer.analyze /home/giacomoschiavo/segments/PROVA/valid -o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad/augm_wabad_Labels.txt -t 16 --combine_results -c /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad/augm_wabad.tflite 

python -m birdnet_analyzer.analyze /home/giacomoschiavo/segments/PROVA/test -o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad/test/ --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad/augm_wabad_Labels.txt -t 16 --combine_results -c /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad/augm_wabad.tflite 

In [22]:
DATASET_VAR = 'augm_wabad_1'
RESULTS_PATH = f'/home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}'
VALID_TABLE = f'{RESULTS_PATH}/valid/BirdNET_SelectionTable.txt'
TEST_TABLE = f'{RESULTS_PATH}/test/BirdNET_SelectionTable.txt'

In [23]:
f'python -m birdnet_analyzer.analyze /home/giacomoschiavo/segments/PROVA/valid -o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}/{DATASET_VAR}_Labels.txt -t 16 --combine_results -c /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}/{DATASET_VAR}.tflite '

'python -m birdnet_analyzer.analyze /home/giacomoschiavo/segments/PROVA/valid -o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad_1/valid --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad_1/augm_wabad_1_Labels.txt -t 16 --combine_results -c /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad_1/augm_wabad_1.tflite '

In [24]:
f'python -m birdnet_analyzer.analyze /home/giacomoschiavo/segments/PROVA/test -o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}/{DATASET_VAR}_Labels.txt -t 16 --combine_results -c /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/{DATASET_VAR}/{DATASET_VAR}.tflite '

'python -m birdnet_analyzer.analyze /home/giacomoschiavo/segments/PROVA/test -o /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad_1/test --slist /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad_1/augm_wabad_1_Labels.txt -t 16 --combine_results -c /home/giacomoschiavo/finetuning-BirdNET/models/finetuned/augm_wabad_1/augm_wabad_1.tflite '

In [25]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/species_dict_map.json') as f:
    species_dict = json.load(f)
inv_species_dict = {value: key for key, value in species_dict.items()}

In [37]:
def get_conf_scores():
    conf_scores = {}

    #  {'Aeroplane': [(np.float32(0.0), False),
    #               (np.float32(1.3937646e-30), False),
    #               (np.float32(1.0654355e-25), False),
    #               (np.float32(0.0), False),

    with open(TEST_TABLE, 'r') as f:
        reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
        for row in reader:
            file_path = row['Begin Path']
            pred_species_name = row['Common Name']
            if row['Common Name'] in inv_species_dict:
                pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
            true_species_name = file_path.split('/')[-2]
            confidence = float(row['Confidence'])
            if pred_species_name not in conf_scores:
                conf_scores[pred_species_name] = []
            is_correct = pred_species_name == true_species_name
            conf_scores[pred_species_name].append((confidence, is_correct))
    return conf_scores
            

In [38]:
conf_scores = get_conf_scores()

In [40]:
from sklearn.metrics import f1_score
import numpy as np 
def compute_best_thresholds(conf_scores, num_thresholds=100, min_thresh=0.1, max_thresh=0.95):
    thresholds = {}

    for species, values in conf_scores.items():
        probs, truths = zip(*values)
        probs = np.array(probs)
        truths = np.array(truths).astype(int)

        best_thresh = 0.1
        best_f1 = 0.0

        for thresh in np.linspace(min_thresh, max_thresh, num_thresholds):
            preds = (probs >= thresh).astype(int)
            f1 = f1_score(truths, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        thresholds[species] = best_thresh
        print(f"📊 {species} -> {best_thresh:.3f}, F1-score: {best_f1:.3f}")

    return thresholds



In [41]:
best_thresholds = compute_best_thresholds(conf_scores)

📊 Erithacus rubecula_European Robin -> 0.950, F1-score: 0.155
📊 Dryocopus martius_Black Woodpecker -> 0.916, F1-score: 0.005
📊 Fringilla coelebs_Common Chaffinch -> 0.950, F1-score: 0.227
📊 Sylvia atricapilla_Eurasian Blackcap -> 0.941, F1-score: 0.224
📊 None -> 0.950, F1-score: 0.874
📊 Troglodytes troglodytes_Eurasian Wren -> 0.950, F1-score: 0.029
📊 Turdus philomelos_Song Thrush -> 0.401, F1-score: 0.041
📊 Phylloscopus collybita_Common Chiffchaff -> 0.950, F1-score: 0.497
📊 Dendrocopos major_Great Spotted Woodpecker -> 0.950, F1-score: 0.010
📊 Lophophanes cristatus_Crested Tit -> 0.830, F1-score: 0.003
📊 Regulus regulus_Goldcrest -> 0.950, F1-score: 0.003
📊 Vegetation -> 0.538, F1-score: 0.012
📊 Regulus ignicapilla_Common Firecrest -> 0.950, F1-score: 0.072
📊 Aeroplane -> 0.100, F1-score: 0.000
📊 Muscicapa striata_Spotted Flycatcher -> 0.332, F1-score: 0.037
📊 Wind -> 0.950, F1-score: 0.072
📊 Periparus ater_Coal Tit -> 0.950, F1-score: 0.028
📊 Loxia curvirostra_Common Crossbill -> 0.

In [43]:
pred_segments = {}
DEFAULT_THRESH = 0.2
#  {'Aeroplane': [(np.float32(0.0), False),
#               (np.float32(1.3937646e-30), False),
#               (np.float32(1.0654355e-25), False),
#               (np.float32(0.0), False),

with open(TEST_TABLE, 'r') as f:
    reader = csv.DictReader(f, delimiter='\t')  # Usa TAB come separatore
    for row in reader:
        file_path = row['Begin Path']
        audio_name = os.path.basename(file_path)    # in test non sono presenti WABAD o augm
        only_audio_name = "_".join(audio_name.split("_")[:2]) + ".WAV"
        segm = "_".join(audio_name.split(".")[0].split("_")[-2:])
        pred_species_name = row['Common Name']
        if row['Common Name'] in inv_species_dict:
            pred_species_name = '_'.join([inv_species_dict[row['Common Name']], row['Common Name']])
        confidence = float(row['Confidence'])
        if only_audio_name not in pred_segments:
            pred_segments[only_audio_name] = {}
        if pred_species_name not in best_thresholds:
            best_thresholds[pred_species_name] = DEFAULT_THRESH
        if confidence >= best_thresholds[pred_species_name]:
            if segm not in pred_segments[only_audio_name]:
                pred_segments[only_audio_name][segm] = []
            pred_segments[only_audio_name][segm].append(pred_species_name)
            

In [45]:
from sklearn.preprocessing import MultiLabelBinarizer

test_species_list = list(best_thresholds.keys())
mlb = MultiLabelBinarizer()
mlb.fit([test_species_list])

len(mlb.classes_)

22

In [46]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/PROVA/true_segments_test.json') as f:
    true_segments_test = json.load(f)

In [48]:
with open('/home/giacomoschiavo/finetuning-BirdNET/utils/PROVA/true_segments_train.json') as f:
    true_segments_full = json.load(f)

In [49]:
true_segments_full.update(true_segments_test)

In [50]:
true_segments = {}
for audio in pred_segments.keys():                         # {'20190608_070000.WAV': {'189_0': ['Poecile palustris_Marsh Tit'],
    true_segments[audio] = {}                                  # {'20190608_070000.WAV' : {}}
    for key, values in pred_segments[audio].items():       #  {'189_0': ['Poecile palustris_Marsh Tit'],
        true_segments[audio][key] = [value if len(value.split("_")) > 1 else value + "_" for value in true_segments_full[os.path.basename(audio)][key] if value in test_species_list]  # {'20190608_070000.WAV': {'189_0': ['Poecile palustris_Marsh Tit'],

In [51]:
y_pred = []
y_true = []
for audio in pred_segments:
    # sort in increasing order
    sortable_true_segments = { str(key): value for key, value in true_segments[audio].items() }
    sortable_pred_segments = { str(key): value for key, value in pred_segments[audio].items() }
    sorted_true_segments = dict(sorted(sortable_true_segments.items()))
    sorted_pred_segments = dict(sorted(sortable_pred_segments.items()))
    y_true.append(mlb.transform(sorted_true_segments.values()))     # apply transform on every label of every segment
    y_pred.append(mlb.transform(sorted_pred_segments.values()))     # apply transform on every label of every segment

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Aeroplane_', 'None_'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['None_'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['None_', 'Wind_'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['None_', 'Vegetation_'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['None_', 'Vegetation_', 'Wind_'] will be ignored
  warnings.warn(
/opt/conda/envs/myenv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['Pecking

In [52]:
from sklearn.metrics import classification_report
import pandas as pd
report = classification_report(y_true, y_pred, target_names=mlb.classes_, zero_division=0, output_dict=True)

report_df = pd.DataFrame(report).T
report_df

,precision,recall,f1-score,support
Aeroplane,0.000000,0.000000,0.000000,0.0
Certhia familiaris_Eurasian Treecreeper,0.000000,0.000000,0.000000,3.0
Dendrocopos major_Great Spotted Woodpecker,0.005336,1.000000,0.010616,25.0
Dryocopus martius_Black Woodpecker,0.002847,1.000000,0.005679,21.0
Erithacus rubecula_European Robin,0.095112,0.967568,0.173198,555.0
Fringilla coelebs_Common Chaffinch,0.144452,0.987132,0.252024,1088.0
Lophophanes cristatus_Crested Tit,0.001689,0.357143,0.003362,14.0
Loxia curvirostra_Common Crossbill,0.016748,0.788462,0.032800,52.0
Muscicapa striata_Spotted Flycatcher,0.021739,0.157407,0.038202,108.0
None,0.000000,0.000000,0.000000,0.0
